In [1]:
import requests as req 
from bs4 import BeautifulSoup, CData, Tag
import time
import tqdm
import pickle
import pandas as pd

In [2]:
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

In [3]:
sites = { 
    'Фонтанка.ру' : ['https://www.fontanka.ru/fontanka.rss', ''],                  
    'LENTA.RU' : ['https://lenta.ru/rss', ''],                  
    'TASS' : ['https://tass.ru/rss/v2.xml', ''],
    'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']           
}

In [4]:
sites

{'Фонтанка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [5]:
def parsing():
    data = { "data" : [] }
        
    for site in range(len(sites)):
        
        url = list(sites.values())[site][0]
                
        resp = req.get(url)

        soup = BeautifulSoup(resp.content, features="xml")

        tag_item = soup.find_all('item')

        for item in tag_item:
            data['data'].append({
                "title": item.title.text,
                "pubDate": item.pubDate.text,
                "site": list(sites.keys())[site],
            })

    return pd.DataFrame(data['data'])

In [6]:
df = parsing()
df.sample(8)

,title,pubDate,site
161,Россиянам назвали вышедшие из моды короткие ст...,"Thu, 19 Jan 2023 17:44:35 +0300",LENTA.RU
42,В офисе Зеленского назвали США лучшим другом У...,"Thu, 19 Jan 2023 23:20:00 +0300",LENTA.RU
389,В Белгородской области сообщили об обстреле се...,"Thu, 19 Jan 2023 12:41:24 +0300",ВЕДОМОСТИ
469,"Лавров заявил, что Россия отстоит правду","Wed, 18 Jan 2023 12:39:49 +0300",ВЕДОМОСТИ
123,В НАТО пожаловались на отсутствие «серебряной ...,"Thu, 19 Jan 2023 19:03:00 +0300",LENTA.RU
215,Пытавшегося вывезти порох на Украину россиянин...,"Thu, 19 Jan 2023 16:21:12 +0300",LENTA.RU
379,Роспотребнадзор оценил идею о запрете вейпов,"Thu, 19 Jan 2023 13:49:53 +0300",ВЕДОМОСТИ
177,Двое иностранцев похитили россиянина из-за дол...,"Thu, 19 Jan 2023 17:20:00 +0300",LENTA.RU


In [7]:
topics = pd.DataFrame(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', 'nan', 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'])

In [8]:
model = None

with (open('../../lenta.ru/models/knb_ppl_clf.pkl', 'rb')) as openfile:
    model = pickle.load(openfile)

In [9]:
predicted_topics = model.predict(df['title'])

In [13]:
df_news = pd.concat([pd.Series(predicted_topics), df], ignore_index=True, axis=1)

df_news = df_news.rename(columns={0: "category", 1: "title", 2: "pub_date", 3: "site"})

df_news['parsed_date'] = None
for index, row in enumerate(df_news['pub_date']):
    parsed = time.strptime(row, "%a, %d %b %Y %H:%M:%S %z")
    df_news['parsed_date'][index] = time.strftime('%Y-%m-%d %H:%M:%S', parsed)

df_news.sample(8)

,category,title,pub_date,site,parsed_date
376,Наука и техника,Борисов ожидает завершения строительства старт...,"Thu, 19 Jan 2023 14:15:50 +0300",ВЕДОМОСТИ,2023-01-19 14:15:50
278,Мир,Госдеп следит за данными об открытии ФСБ дела ...,"Thu, 19 Jan 2023 23:10:43 +0300",TASS,2023-01-19 23:10:43
126,Экономика,Модель сделала татуировку на глазном яблоке и ...,"Thu, 19 Jan 2023 18:54:00 +0300",LENTA.RU,2023-01-19 18:54:00
230,Спорт,"""Манчестер Сити"" обыграл ""Тоттенхэм"" в матче ч...","Fri, 20 Jan 2023 00:54:07 +0300",TASS,2023-01-20 00:54:07
6,Из жизни,Сумеете ли вы отличить Мрака от Сумрака? Ленин...,"Thu, 19 Jan 2023 22:53:59 +0300",Фонтанка.ру,2023-01-19 22:53:59
501,Россия,Песков: Путин не будет объявлять о новой мобил...,"Tue, 17 Jan 2023 20:25:05 +0300",ВЕДОМОСТИ,2023-01-17 20:25:05
108,Интернет и СМИ,Посылка с грязными ботинками из интернет-магаз...,"Thu, 19 Jan 2023 19:32:00 +0300",LENTA.RU,2023-01-19 19:32:00
59,Экономика,Сербия поможет Украине оборудованием для энерг...,"Thu, 19 Jan 2023 21:55:00 +0300",LENTA.RU,2023-01-19 21:55:00


In [14]:
import json
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

def write_to_kafka(df):
    hm = {}
    for _, row in df.iterrows():
        hm["category"] = row["category"]
        hm["title"] = row["title"]
        hm["site"] = row["site"]
        hm["pub_date"] = row["parsed_date"]        
        producer.send('foobar', json.dumps(hm).encode('utf-8'))
        producer.flush()

write_to_kafka(df_news)